In [14]:
#install python (and check the add to path option during installation), then install the ema_workbench python module.
#this can be done by executing the following command in the command prompt:
# pip install ema_workbench

#import necessary python libraries for the analysis
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

#import EMA workbench tools
from ema_workbench.em_framework import samplers, evaluators, util
from ema_workbench import (RealParameter, ScalarOutcome, Model, Constant, TimeSeriesOutcome)
from ema_workbench import (perform_experiments, ema_logging, save_results, 
                           load_results, SequentialEvaluator, MultiprocessingEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)


#import PRIM algorithm
from ema_workbench.analysis import prim

In [3]:
# read the benefits data gathered from the literature
b = pd.read_csv("Sample_benefit_data.csv")
display(b)

# read the costs and benefits data gathered from the literature
c = pd.read_csv("Sample_cost_data.csv")
display(c)

,Item,0,1,2,3,4,5,6,7,8
0,UHIE_mitigation,0.010000,0.030000,0.040000,0.200000,1.000000,0.040000,0.010000,0.020000,0.050000
1,AQ_reduction,0.004500,0.013500,0.018000,0.090000,0.450000,0.018000,0.004500,0.009000,0.022500
2,AQ_reduction,0.002025,0.006075,0.008100,0.040500,0.202500,0.008100,0.002025,0.004050,0.010125
3,UHIE_mitigation,0.000911,0.002734,0.003645,0.018225,0.091125,0.003645,0.000911,0.001823,0.004556
4,UHIE_mitigation,0.000410,0.001230,0.001640,0.008201,0.041006,0.001640,0.000410,0.000820,0.002050
5,B_enhancement,0.000185,0.000554,0.000738,0.003691,0.018453,0.000738,0.000185,0.000369,0.000923
6,B_enhancement,0.000083,0.000249,0.000332,0.001661,0.008304,0.000332,0.000083,0.000166,0.000415
7,B_enhancement,0.000037,0.000112,0.000149,0.000747,0.003737,0.000149,0.000037,0.000075,0.000187


,Item,0,1,2,3,4,5,6,7,8
0,Installation,10,0,0,0,0,0,0,0,0
1,Installation,9,0,0,0,0,0,0,0,0
2,Maintenance,0,0,5,0,0,5,0,0,5
3,Maintenance,0,2,2,2,2,2,2,2,2


In [57]:
def cba(r = 3, 
       incentive = 0,
       time_horizon = 9):
    npv = np.sum([1,2])
    npv2 = np.sum([3,4])
    return npv, npv2

In [58]:
# specify the EMA workbench's wrapper around the CBA model function, so as to perform 
# exploratory modelling within it

# instantiate the model
cba_model = Model('cba', function=cba)
# cba_model.time_horizon = 9

# specify the uncertainties
uncertainties = []
# for benefit in b_min.index:
#     for i in b_min.columns:
#         if b_max.loc[benefit,str(i)] > b_min.loc[benefit,str(i)]:
#             uncertainties.append(RealParameter(benefit+"_"+i+"b", b_min.loc[benefit,str(i)], b_max.loc[benefit,str(i)]))
        
# for cost in c_min.index:
#     for i in c_min.columns:
#         if c_max.loc[cost,str(i)] > c_min.loc[cost,str(i)]:
#             uncertainties.append(RealParameter(cost+"_"+i+"c", c_min.loc[cost,str(i)], c_max.loc[cost,str(i)]))
    
uncertainties.append(RealParameter('r', 3, 5))
cba_model.uncertainties = uncertainties

# set the lever
cba_model.levers = [RealParameter("incentive", 0, 10)]

#specify outcomes 
cba_model.outcomes = [ScalarOutcome('npv'),ScalarOutcome("npv2")]

# override some of the defaults of the model
cba_model.constants = [Constant('time_horizon', 9)]

In [59]:
from ema_workbench import SequentialEvaluator

with SequentialEvaluator(cba_model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=10)

[MainProcess/INFO] performing 10 scenarios * 1 policies * 1 model(s) = 10 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 1 cases completed
[MainProcess/INFO] 2 cases completed
[MainProcess/INFO] 3 cases completed
[MainProcess/INFO] 4 cases completed
[MainProcess/INFO] 5 cases completed
[MainProcess/INFO] 6 cases completed
[MainProcess/INFO] 7 cases completed
[MainProcess/INFO] 8 cases completed
[MainProcess/INFO] 9 cases completed
[MainProcess/INFO] 10 cases completed
[MainProcess/INFO] experiments finished


In [33]:
outcomes

{'npv': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'npv2': array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])}